# 画像判別機

## Google driveとColabの連携

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# なぜかうまくいかへん
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

## ファイルの手動アップロード

In [0]:
from google.colab import files
uploaded = files.upload()

## ファイルから、numpy配列に格納

In [0]:
X = np.load("X_data.npy")
Y = np.load('Y_data.npy')

## ライブラリの import

In [0]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf

In [0]:
import keras
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, list_pictures, load_img
from sklearn.model_selection import train_test_split

## プロット時に羅線を削除する関数

In [0]:
def clearLabel(_ax):
  _ax.tick_params(labelbottom="off",bottom="off")
  _ax.tick_params(labelleft="off",left="off")
  _ax.set_xticklabels([]) 
  _ax.axis('off')
  return _ax

In [0]:
# test plot
fig,ax = plt.subplots(dpi=40)
clearLabel(ax)
plt.imshow(X[0])
plt.show()

## trainデータとtestデータへ分ける

In [0]:
# 学習用データとテストデータ
X_train, X_test, Y_train, Y_test = train_test_split(X , Y, test_size=0.33, random_state=111)

In [0]:
# check train data
for i in range(100,110):
  fig,ax = plt.subplots(dpi=40)
  clearLabel(ax)
  print(Y_train[i])
  plt.imshow(X_train[i])
  plt.show()

# モデルの構築

## 各パラメーターの設定

In [0]:
#何回回すかの決定
MAX_EPOCH = 1_500
# サンプリングの数を設定
BATCH_SIZE = 32

## CNNの構築

In [0]:
# CNNを構築
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))       # クラスは3個
model.add(Activation('softmax'))

## モデルのコンパイル

In [0]:
# optimizer の adam の設定
adam = keras.optimizers.Adam(lr=1e-3)

# optimizerにはadamを指定
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

## 実行

In [0]:
history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=MAX_EPOCH,
                   validation_data = (X_test, Y_test), verbose = 1)

## 結果のプロット

# 予測

## 予測結果のプロット

In [0]:
# ラベルの作成
category = ["新垣結衣","有村架純","本田翼"]

# ランダム関数
import random
def randam_data(data_set, label, batch_size) :
    index = np.random.choice(len(data_set),batch_size)
    data_set = data_set[index]
    label = label[index]
    return data_set, label
  
test_x, test_y = randam_data(X_test, Y_test, 5)
pre_ram = model.predict(test_x)

In [0]:
# plot
for (i, pre) in enumerate(pre_ram):
  y = pre.argmax()
  y_ = test_y[i].argmax()
  print("予測 :{0}  {1}% ".format(category[y], round(pre[y]*100)) )
  print("正解: ", category[y_])
  fix, ax = plt.subplots(dpi=30)
  clearLabel(ax)
  plt.imshow(test_x[i])
  plt.show()

## 他の画像をアップロードして予測！

In [0]:
uploaded = files.upload()
!ls

In [0]:
img = cv2.imread("cat.JPG")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_ = cv2.resize(img, (56, 56))
img_ = [img_ , img_]
img_ = np.array(img_)

img_= img_.astype('float32')
img_ = img_ / 255.0

predict = model.predict(img_)
pre = predict[0]
y = pre.argmax()
print("予測 :{0}  {1}% ".format(category[y], round(pre[y]*100)) )
print("新垣結衣: {0}%, 有村架純: {1}%, 本田翼: {2}%".format(round(pre[0]*100),round(pre[1]*100),round(pre[2]*100)))
fix, ax = plt.subplots(dpi=30)
clearLabel(ax)
plt.imshow(img_[1])
plt.show()